Домашнее задание по теме «Свёрточные сети»<br>
Цель задания:<br>
 научиться базово подбирать архитектуру, строить и обучать свёрточные нейронные сети для решения задачи классификации.
<br><br>
Контекст
Это домашнее задание по сути является продолжением предыдущего. В нём вы меняете архитектуру сети с полносвязной на свёрточную.
<br><br>
Задание<br>
Постройте модель, используя архитектуру со свёрточными слоями, для классификации Fashion MNIST. Итоговое качество (accuracy) должно быть не ниже 89,5.
<br><br>
Инструкция к выполнению задания
<br><br>
Скачайте тренировочную и тестовою часть датасета Fashion <br>
Постройте архитектуру модели, используя свёрточные слои, слои регуляризации и один финальный полносвязный слой<br>
Обучите модель до необходимого качества. <br>Если сеть не обучается до необходимого качества, попробуйте поменять архитектуру сети, варьируя параметры свёрток, количество каналов, количество свёрточных слоёв, слои регуляризации, тип оптимайзера и т. д.<br><br>
Формат сдачи работы<br>
Прикрепите ссылку на готовое решение в личном кабинете. Работу можно отправлять в виде ссылки на python-ноутбук из GitHub, Google Colaboratory или аналогичных платформ. Не забудьте открыть доступ на просмотр и комментирование.<br><br>

Критерии оценивания<br>
По итогу выполнения задания вы получите зачёт.<br>

Задание считается выполненным, если:<br>

корректно получены трейн и тест части<br>
сеть обучается<br>
итоговое качество на тестовой выборке выше указанного в задании<br>


In [ ]:
import pandas as pd
import numpy as np

import time


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchsummary import summary

import torchvision as tv

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
BATCH_SIZE=256

In [ ]:
train_dataset = tv.datasets.FashionMNIST(root = '.', train = True, download=True, transform=tv.transforms.ToTensor())
test_dataset = tv.datasets.FashionMNIST(root = '.', train = False, download=True, transform=tv.transforms.ToTensor())

100%|██████████| 26421880/26421880 [00:02<00:00, 13148731.01it/s]


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 211229.19it/s]


Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3875827.37it/s]


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 12359631.94it/s]

Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



In [ ]:
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [ ]:
28*28

784

In [ ]:
model_x = torch.nn.Sequential(
    torch.nn.Flatten(), # вытягиваем в вектор
    torch.nn.Linear(784, 256), # полносвязный слой с входом 28х28=784 и выходом 256
    torch.nn.ReLU(), # функция активации
    torch.nn.Linear(256, 10) # полносвязный слой с входом 256 и выходом 10 - каждый нейрон отвечает за свою цифру/класс
)

In [ ]:
summary(model_x, input_size=(1, 28, 28), batch_size=BATCH_SIZE, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                 [256, 784]               0
            Linear-2                 [256, 256]         200,960
              ReLU-3                 [256, 256]               0
            Linear-4                  [256, 10]           2,570
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.77
Forward/backward pass size (MB): 2.55
Params size (MB): 0.78
Estimated Total Size (MB): 4.09
----------------------------------------------------------------


<font color=green size=3>
<br>
В описании, что выше, плюс в том, что и функции, как и слои, отображаются в распечатке модели.
<br>
Но если функции будут более замысловатые, они не отобразятся слоями, и возможно только запутают.
<br>
Функции лучше смотреть в методе forward.
</font>

In [ ]:
class FshnLinF(nn.Module):
    def __init__(self):
        #super(FshnLinF, self).__init__()
        super().__init__()

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 256, bias=True)
        self.fc2 = nn.Linear(256, 10, bias=True)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

<font color=green size=3>
<br>
Будем использовать такое описание модели, как с одной стороны краткое, а с другой стороны - гибкое.
<br>
<br>
</font>

In [ ]:
model = FshnLinF()
print(model)

FshnLinF(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [ ]:
BATCH_SIZE

256

In [ ]:
summary(model, input_size=(1, 28, 28), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 256]         200,960
            Linear-3                   [-1, 10]           2,570
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.78
Estimated Total Size (MB): 0.79
----------------------------------------------------------------


In [ ]:
loss = torch.nn.CrossEntropyLoss() # функция потерь для классификации
trainer = torch.optim.SGD(model.parameters(), lr=.01) # оптимайзер функции потерь (передали туда параметры и шаг обучения) - схокастический градиентный спуск
num_epochs = 10 # кол-во эпох

In [ ]:
 # обучаем модель
def train_model():
    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0 # инициализируем параметры для процесса обучения
        train_loss, train_acc = 0., 0.
        start=time.time() # фиксируем время начала обучения

        model.train() # переводим модель в режим обучения (чтобы использовать инструментарий типа dropout и bacth normalization)
        for X, y in train:  # итерируемся по данным
            trainer.zero_grad() # обнуляем старые значения градиентов
            y_pred = model(X) # прогоняем данные через модель, получаем предсказания
            l = loss(y_pred, y) # вызываем функцию потерь для получения и оценки ошибки (сравниваем с истинными размеченными значениями y - позиция 1 в строке от 0 до 9)
            l.backward() # считаем градиенты
            trainer.step() # делаем шаг градиентного спуска
            train_loss += l.item() # суммируем среднее значение функции потерь на каждом элементе этого батча в этой иттерации
            train_acc += (y_pred.argmax(dim=1) == y).sum().item() # находим максимальную (по предсказанной вероятности) позицию в предсказании и сравниваем с позицией верного ответа
            # суммируем матрицу и получаем кол-во правильно предсказанных символов - это и есть почти аккураси в этом случае
            # т.к. True=1 а False=0
            train_iters += 1 # сохраняем кол-во иттераций
            train_passed += len(X) # сохраняем, сколько данных прогнали через модель

 # делаем такой же цикл для тестовых данных
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval() # переводим в режим исполнения
        for X, y in test:
           # не обнуляем старые значения градиентов
            y_pred = model(X)
            l = loss(y_pred, y)
             # не считаем градиенты и не делаем шаг градиентного спуска
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
 # выводим промежуточные результаты за эпоху
        print("ep: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )

In [ ]:
#%%time
#train_model()

<font color=green size=3>
Получили вполне приличный результат
<br>
train_acc: 0.80975, test_acc: 0.7988
<br>
на самой простейшей сети из полносвязных слоев
<br>
<br>
Теперь переделаем модель с использованием сверточных слоев
<br>
Добавим два сверточных слоя, а полносвязную голову оставим для классификации
<br>
<br>
</font>

In [ ]:
class FshnConLinF(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

    self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)

  def forward(self, x):
    # conv 1
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    # conv 2
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    # fc1
    x = x.reshape(-1, 12*4*4)
    x = self.fc1(x)
    x = F.relu(x)

    # fc2
    x = self.fc2(x)
    x = F.relu(x)

    # output
    x = self.out(x)
    # don't need softmax here since we'll use cross-entropy as activation.
    # наша функция потерь- кросэнтропия, которая и применяет внутри себя функцию софтмакс при расчете ошибки для расчета градиентов
    return x

<font color=green size=3>
Схема такая:
<br>
256 - наш размер батча, но дальше  пишем все для одного имиджа
<br>
1@28x28 - разм. входящего имиджа
<br>
6@24x24 - первая свертка с ядром 5 (результат)
<br>
6@12x12 - первый макспуллинг паддинг 2 шаг 2
<br>
12@8x8 - вторая свертка с ядром 5
<br>
12@4x4 - второй макспуллинг паддинг 2 шаг 2
<br>
12@4x4 решейпим (reshape) в 1x192 на входе - первый полносвязный слой (обошлись без Flatten и без MaxPulling в 1x1), получили 1x120 на выходе
<br>
1x120 -> 1x60 - второй полносвязны слой
<br>
1x60 -> 1x10 - третий полносвязный слой с выходом 10 классов
<br>

<br>
<br>
</font>

In [ ]:
model = FshnConLinF()
print(model)

FshnConLinF(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)


In [ ]:
summary(model, input_size=(1, 28, 28), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
            Conv2d-2             [-1, 12, 8, 8]           1,812
            Linear-3                  [-1, 120]          23,160
            Linear-4                   [-1, 60]           7,260
            Linear-5                   [-1, 10]             610
Total params: 32,998
Trainable params: 32,998
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 0.13
Estimated Total Size (MB): 0.16
----------------------------------------------------------------


In [ ]:
loss = torch.nn.CrossEntropyLoss() # функция потерь для классификации
trainer = torch.optim.SGD(model.parameters(), lr=.01) # оптимайзер функции потерь (передали туда параметры и шаг обучения) - схокастический градиентный спуск
num_epochs = 10 # кол-во эпох

In [ ]:
#%%time
#train_model()

<font color=green size=3>
 Результат комбинированной модели слабее,чем был только на двух полносвязных слоях.
<br>
было: train_acc: 0.80975, test_acc: 0.7988
<br>
стало: train_acc: 0.71612, test_acc: 0.7266
<br>
<br>
Плюсом можно считать равномерные результаты на обучающей и тестовой выборках.
<br>
Возможно при длительном обучении это позволит лучше  обучиться.
<br>
Видим, что число параметров кардинально (на порядок) сократилось, а время обучения в два раза увеличилось. Памяти расходуется меньше, но расчетов требуется больше.
<br>
Посмотрим, что изменится, если перенести расчеты на GPU
<br>
<br>
</font>

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" # переключаемся, если доступно
device

'cuda'

In [ ]:
def train_device(net, train_iter, test_iter, optimizer, num_epochs, device):
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for X, y in train_iter:
            X, y = X.to(device), y.to(device) # перекладываем данные на GPU
            optimizer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

        test_acc = evaluate_accuracy_device(test_iter, net, device)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}' \
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

In [ ]:
def evaluate_accuracy_device(data_iter, net, device):  # передаем текущее устройство
    acc_sum, n = torch.Tensor([0]).to(device), 0  # переводим на него тензор аккураси
    for X, y in data_iter:
        X, y = X.to(device), y.to(device) # переводим на него тензора данных
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [ ]:
model = FshnConLinF()

In [ ]:
model = model.to(device) # перекладываем модель на GPU

In [ ]:
%%time
loss = torch.nn.CrossEntropyLoss() # функция потерь для классификации
lr, num_epochs = 0.01, 20
trainer = torch.optim.SGD(model.parameters(), lr=lr)
#train_device(model, train, test, trainer, num_epochs, device)

CPU times: user 402 µs, sys: 0 ns, total: 402 µs
Wall time: 410 µs


<font color=green size=3>
Скорость расчета на GPU возросла примерно в 2 раза.
<br>
Считается чуть быстрее, чем считались 2 полносвязных слоя на CPU. Т.е. очень приемлемо.
<br>
было: train_acc: 0.80975, test_acc: 0.7988
<br>
было при 10 эпохах: train_acc: 0.71612, test_acc: 0.7266
<br>
стало при 20 эпохах: train acc 0.800, test acc 0.797
<br>
<br>
Потенциал обучения по прежнему хороший. На 20 эпохах результат догнал целиком полносвязную сеть и может обучаться дальше.
<br>
Попробуем за счет добавления дополнительных сверток/каналов/параметров поднять скорость обучения.
<br>

<br>
<br>
</font>

In [ ]:
class FshnConPlusLinF(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=1)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding=1)
    self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)

    self.fc1 = nn.Linear(in_features=120, out_features=90)
    self.fc2 = nn.Linear(in_features=90, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)

  def forward(self, x):
    # conv 1
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    # conv 2
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    # conv 3
    x = self.conv3(x)

    # fc1
    x = x.reshape(-1, 120)
    x = self.fc1(x)
    x = F.relu(x)

    # fc2
    x = self.fc2(x)
    x = F.relu(x)

    # output
    x = self.out(x)
    # don't need softmax here since we'll use cross-entropy as activation.
    return x

In [ ]:
model1 = FshnConPlusLinF()
print(model1)

FshnConPlusLinF(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=90, bias=True)
  (fc2): Linear(in_features=90, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)


In [ ]:
summary(model1, input_size=(1, 28, 28), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 26, 26]             156
            Conv2d-2           [-1, 16, 11, 11]           2,416
            Conv2d-3            [-1, 120, 1, 1]          48,120
            Linear-4                   [-1, 90]          10,890
            Linear-5                   [-1, 60]           5,460
            Linear-6                   [-1, 10]             610
Total params: 67,652
Trainable params: 67,652
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.26
Estimated Total Size (MB): 0.31
----------------------------------------------------------------


# <font color=green size=3>
За счет вставленного 3 сверточного слоя подняли кол-во каналов до 120 и кол-во параметров с 33 до 67 тысяч.
<br>
Возможно сеть это улучшило, но на скорости обучения никак не сказалось.
<br>
</font>

In [ ]:
model1 = model1.to(device) # перекладываем модель на GPU

In [ ]:
%%time
loss = torch.nn.CrossEntropyLoss() # функция потерь для классификации
lr, num_epochs = 0.01, 10
trainer = torch.optim.SGD(model1.parameters(), lr=lr)
#train_device(model1, train, test, trainer, num_epochs, device)

CPU times: user 628 µs, sys: 0 ns, total: 628 µs
Wall time: 556 µs


In [ ]:
class FshnConPlus2LinF(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=1)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding=1)
    self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)

    self.bn1 = nn.BatchNorm1d(120)
    self.fc1 = nn.Linear(in_features=120, out_features=90)
    self.fc2 = nn.Linear(in_features=90, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)


  def forward(self, x):
    # conv 1
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    # conv 2
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    # conv 3
    x = self.conv3(x)

    # fc1
    x = x.reshape(-1, 120)
    x = self.bn1(x)
    x = self.fc1(x)
    x = F.relu(x)

    # fc2
    x = self.fc2(x)
    x = F.relu(x)

    # output
    x = self.out(x)
    # don't need softmax here since we'll use cross-entropy as activation.
    return x

In [ ]:
model2 = FshnConPlus2LinF()
print(model2)

FshnConPlus2LinF(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=120, out_features=90, bias=True)
  (fc2): Linear(in_features=90, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)


In [ ]:
model2 = model2.to(device) # перекладываем модель на GPU

In [ ]:
summary(model2, input_size=(1, 28, 28), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 26, 26]             156
            Conv2d-2           [-1, 16, 11, 11]           2,416
            Conv2d-3            [-1, 120, 1, 1]          48,120
       BatchNorm1d-4                  [-1, 120]             240
            Linear-5                   [-1, 90]          10,890
            Linear-6                   [-1, 60]           5,460
            Linear-7                   [-1, 10]             610
Total params: 67,892
Trainable params: 67,892
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.26
Estimated Total Size (MB): 0.31
----------------------------------------------------------------


In [ ]:
%%time
loss = torch.nn.CrossEntropyLoss() # функция потерь для классификации
lr, num_epochs = 0.01, 10
trainer = torch.optim.SGD(model2.parameters(), lr=lr)
#train_device(model2, train, test, trainer, num_epochs, device)

CPU times: user 0 ns, sys: 319 µs, total: 319 µs
Wall time: 317 µs


<font color=green size=3>
Между сверточными слоями и головой добавили 1 слой нормализации и скорость обучения возросла кардинально.
<br>
полносвязный вариант
<br>
было: train_acc: 0.80975, test_acc: 0.7988
<br>
комбинированный вариант с 2 сверточными слоями
<br>
было: при 10 эпохах: train_acc: 0.71612, test_acc: 0.7266
<br>
было: при 20 эпохах: train acc 0.800, test acc 0.0.797
<br>
комбинированный с 3 сверточными слоями
<br>
было: при 10 эпохах: train acc 0.719, test acc 0.693
<br>
комбинированный с 3 сверточными слоями и нормализацией
<br>
стало: при 10 эпохах acc 0.881, test acc 0.873
<br>
<br>
<br>
Потенциал обучения по прежнему хороший.
<br>
Уберем лишние полносвязные слои.
<br>
<br>
</font>

In [ ]:
class FshnConPlus3LinF(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=1)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding=1)
    self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)

    self.bn1 = nn.BatchNorm1d(120)
    self.out = nn.Linear(in_features=120, out_features=10)


  def forward(self, x):
    # conv 1
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    # conv 2
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    # conv 3
    x = self.conv3(x)

    # output
    x = x.reshape(-1, 120)
    x = self.bn1(x)
    x = self.out(x)
    # don't need softmax here since we'll use cross-entropy as activation.
    return x

In [ ]:
model3 = FshnConPlus3LinF()
print(model3)

FshnConPlus3LinF(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=120, out_features=10, bias=True)
)


In [ ]:
model3 = model3.to(device) # перекладываем модель на GPU

In [ ]:
summary(model3, input_size=(1, 28, 28), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 26, 26]             156
            Conv2d-2           [-1, 16, 11, 11]           2,416
            Conv2d-3            [-1, 120, 1, 1]          48,120
       BatchNorm1d-4                  [-1, 120]             240
            Linear-5                   [-1, 10]           1,210
Total params: 52,142
Trainable params: 52,142
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.20
Estimated Total Size (MB): 0.25
----------------------------------------------------------------


In [ ]:
%%time
loss = torch.nn.CrossEntropyLoss() # функция потерь для классификации
lr, num_epochs = 0.01, 10
trainer = torch.optim.SGD(model3.parameters(), lr=lr)
#train_device(model3, train, test, trainer, num_epochs, device)

CPU times: user 265 µs, sys: 79 µs, total: 344 µs
Wall time: 352 µs


<font color=green size=3>
Странно, но удаление двух полносвязных слоев и всех их функций активации похоже никак не сказалось на скорости обучения.
<br>
Видимо 3 сверточных слоев и слоя нормализации уже хватает что бы все спорядковать и на долю полносвязных слоев "работы" уже не остается.
<br>
Скорость выполнения похоже зависит от загрузки Гугла и не является показателем, т.к. должна была возрасти из-за уменьшения параметров, а по факту упала.
<br>
<br>
вставим дополнительные слои нормализации между слоями сверток.
<br>
</font>

In [ ]:
class FshnConPlus4LinF(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=1)
    self.bn1 = nn.BatchNorm2d(6)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding=1)
    self.bn2 = nn.BatchNorm2d(16)
    self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)

    self.bn3 = nn.BatchNorm1d(120)
    self.out = nn.Linear(in_features=120, out_features=10)


  def forward(self, x):
    # conv 1
    #x = self.conv1(x)
    x = self.bn1(self.conv1(x))
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    # conv 2
    #x = self.conv2(x)
    x = self.bn2(self.conv2(x))
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    # conv 3
    x = self.conv3(x)

    # output
    x = x.reshape(-1, 120)
    x = self.bn3(x)
    x = self.out(x)
    # don't need softmax here since we'll use cross-entropy as activation.
    return x

In [ ]:
model4 = FshnConPlus4LinF()
print(model4)

FshnConPlus4LinF(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (bn3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=120, out_features=10, bias=True)
)


In [ ]:
model4 = model4.to(device) # перекладываем модель на GPU

In [ ]:
summary(model4, input_size=(1, 28, 28), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 26, 26]             156
       BatchNorm2d-2            [-1, 6, 26, 26]              12
            Conv2d-3           [-1, 16, 11, 11]           2,416
       BatchNorm2d-4           [-1, 16, 11, 11]              32
            Conv2d-5            [-1, 120, 1, 1]          48,120
       BatchNorm1d-6                  [-1, 120]             240
            Linear-7                   [-1, 10]           1,210
Total params: 52,186
Trainable params: 52,186
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.09
Params size (MB): 0.20
Estimated Total Size (MB): 0.30
----------------------------------------------------------------


In [ ]:
%%time
loss = torch.nn.CrossEntropyLoss() # функция потерь для классификации
lr, num_epochs = 0.01, 10
trainer = torch.optim.SGD(model4.parameters(), lr=lr)
train_device(model4, train, test, trainer, num_epochs, device)

epoch 1, loss 0.0028, train acc 0.773, test acc 0.817, time 8.3 sec
epoch 2, loss 0.0019, train acc 0.842, test acc 0.842, time 9.3 sec
epoch 3, loss 0.0016, train acc 0.859, test acc 0.854, time 10.6 sec
epoch 4, loss 0.0015, train acc 0.870, test acc 0.863, time 9.2 sec
epoch 5, loss 0.0014, train acc 0.877, test acc 0.868, time 9.0 sec
epoch 6, loss 0.0013, train acc 0.882, test acc 0.873, time 9.0 sec
epoch 7, loss 0.0013, train acc 0.886, test acc 0.875, time 8.2 sec
epoch 8, loss 0.0012, train acc 0.890, test acc 0.878, time 9.0 sec
epoch 9, loss 0.0012, train acc 0.894, test acc 0.880, time 9.0 sec
epoch 10, loss 0.0012, train acc 0.896, test acc 0.879, time 8.1 sec
CPU times: user 1min 27s, sys: 350 ms, total: 1min 27s
Wall time: 1min 29s


<font color=green size=3>
Добавление двух слоев нормализации между сверточными слоями увеличила скорость обучения.
<br>
после 6 эпохи уже получили
<br>
train acc 0.882, test acc 0.873, time 9.0 sec
<br>
а после 10 эпохи
<br>
train acc 0.896, test acc 0.879, time 8.1 sec
<br>
Попробуем заменить первый сверточный слой с ядром 5х5 на 2 слоя 3х3 и еще добавить каналов.
<br>

</font>

In [ ]:
class FshnConPlus5LinF(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv11 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1)
    self.conv12 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, padding=0)

    self.bn1 = nn.BatchNorm2d(20)
    self.conv2 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=5, padding=1)
    self.bn2 = nn.BatchNorm2d(30)
    self.conv3 = nn.Conv2d(in_channels=30, out_channels=240, kernel_size=5)

    self.bn3 = nn.BatchNorm1d(240)
    self.out = nn.Linear(in_features=240, out_features=10)


  def forward(self, x):
    # conv 1
    x = self.conv11(x)
    x = self.bn1(self.conv12(x))
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    # conv 2
    #x = self.conv2(x)
    x = self.bn2(self.conv2(x))
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)

    # conv 3
    x = self.conv3(x)

    # output
    x = x.reshape(-1, 240)
    x = self.bn3(x)
    x = self.out(x)
    # don't need softmax here since we'll use cross-entropy as activation.
    # наша функция потерь- кросэнтропия, которая и применяет внутри себя функцию софтмакс при расчете ошибки для расчета градиентов
    return x

In [ ]:
model5 = FshnConPlus5LinF()
print(model5)

FshnConPlus5LinF(
  (conv11): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv12): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(20, 30, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(30, 240, kernel_size=(5, 5), stride=(1, 1))
  (bn3): BatchNorm1d(240, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=240, out_features=10, bias=True)
)


In [ ]:
model5 = model5.to(device) # перекладываем модель на GPU

In [ ]:
summary(model5, input_size=(1, 28, 28), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
            Conv2d-2           [-1, 20, 26, 26]           1,820
       BatchNorm2d-3           [-1, 20, 26, 26]              40
            Conv2d-4           [-1, 30, 11, 11]          15,030
       BatchNorm2d-5           [-1, 30, 11, 11]              60
            Conv2d-6            [-1, 240, 1, 1]         180,240
       BatchNorm1d-7                  [-1, 240]             480
            Linear-8                   [-1, 10]           2,410
Total params: 200,180
Trainable params: 200,180
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.33
Params size (MB): 0.76
Estimated Total Size (MB): 1.09
----------------------------------------------------------------


In [ ]:
%%time
loss = torch.nn.CrossEntropyLoss() # функция потерь для классификации
lr, num_epochs = 0.01, 50
trainer = torch.optim.SGD(model5.parameters(), lr=lr)
train_device(model5, train, test, trainer, num_epochs, device)

epoch 1, loss 0.0023, train acc 0.810, test acc 0.850, time 11.1 sec
epoch 2, loss 0.0015, train acc 0.869, test acc 0.868, time 10.9 sec
epoch 3, loss 0.0013, train acc 0.883, test acc 0.875, time 10.6 sec
epoch 4, loss 0.0012, train acc 0.893, test acc 0.882, time 11.0 sec
epoch 5, loss 0.0011, train acc 0.899, test acc 0.886, time 11.0 sec
epoch 6, loss 0.0011, train acc 0.904, test acc 0.890, time 11.0 sec
epoch 7, loss 0.0010, train acc 0.908, test acc 0.891, time 10.7 sec
epoch 8, loss 0.0010, train acc 0.913, test acc 0.895, time 10.5 sec
epoch 9, loss 0.0009, train acc 0.915, test acc 0.895, time 11.0 sec
epoch 10, loss 0.0009, train acc 0.918, test acc 0.896, time 11.2 sec
epoch 11, loss 0.0009, train acc 0.921, test acc 0.899, time 11.0 sec
epoch 12, loss 0.0009, train acc 0.924, test acc 0.899, time 10.8 sec
epoch 13, loss 0.0008, train acc 0.926, test acc 0.900, time 10.5 sec
epoch 14, loss 0.0008, train acc 0.928, test acc 0.901, time 11.0 sec
epoch 15, loss 0.0008, train 

<font color=green size=3>
Для новой архитектуры с 4 сверточными слоями обучение до
<br>
train_acc: 0.883, test_acc: 0.875
<br>
наступает на 3 эпохе.
<br>
обучение до зачетного уровня
<br>
train acc 0.915, test acc 0.895
<br>
наступает на 9 эпохе.
<br>
а с 12 эпохи наступает переобучение
<br>
На расчет 10 эпох на GPU уходит 2 минуты.
<br>
<br>
Если полносвязные слои - это "прокачанная" апроксимация первого порядка из мира традиционной науки, то сверточные методы это первая ласточка понастоящему нейронных подходов, делающих шаг в сторону от проторенной дорожки традиций и привычек.
</font>


|Архитектура|эпоха|train acc|test acc|
|:------------------------|:---|:-------|:--------|
|полносвязный вариант|10|0.80975|0.7988|
|комбинированный вариант с 2 сверточными слоями|10|0.71612|0.7266|
|комбинированный вариант с 2 сверточными слоями|20|0.800|0.797|
|комбинированный вариант с 3 сверточными слоями|10|0.719|0.693|
|комбинированный с 3 сверточными слоями и 1 нормализацией|10|0.881|0.873|
|комбинированный с 3 сверточными слоями c 3 нормализациями|6|0.882|0.873|
|комбинированный с 3 сверточными слоями c 3 нормализациями|10|0.896|0.879|
|комбинированный с 3 сверточными слоями c 4 нормализациями|3|0.883|0.875|
|комбинированный с 3 сверточными слоями c 4 нормализациями|9|0.915|0.895|
|комбинированный с 3 сверточными слоями c 4 нормализациями|10|0.918|0.896|
|||||
|||||
<font color=green size=3>
<br>
</font>

<font color=green size=3>
Это шаблон примечания
<br>
<br>
</font>

первая строка  
вторая строка  
два пробела и ентер  
